# Using LangChain To Create ChatBot Via Konko

## Install necessary Python packages

In [1]:
# !pip install langchain
# !pip install konko

## Import Libraries

In [2]:
import konko
from langchain.chat_models import ChatKonko
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import initialize_agent

##  Set Environment Variables

Next, you need to set your environment variables for KONKO_API_KEY.

1. You can set environment variables for 
   1. KONKO_API_KEY (Required)
   2. OPENAI_API_KEY (Optional)
2. In your current shell session, use the export command:

```shell
export KONKO_API_KEY={your_KONKO_API_KEY_here}
export OPENAI_API_KEY={your_OPENAI_API_KEY_here} #Optional
```

Alternatively, you can add the above lines directly to your shell startup script (such as .bashrc or .bash_profile for Bash shell and .zshrc for Zsh shell) to have them set automatically every time a new shell session starts.



## Set Up Your Language Model

Next, configure your language model. In this context, we'll be utilizing Konko as a provider within the langchain library, specifically using the 'llama-2-13b-chat' model.

In [10]:
llm = ChatKonko(model='meta-llama/Llama-2-13b-chat-hf', max_tokens = 1000)

## Initializing an Conversational Agent


We first need to initialize the agent. Conversational agents require several things such as conversational memory, access to tools, and an llm (which we have already initialized).

In [11]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)
tools = load_tools(["llm-math"], llm=llm)

##  Define the OutputParser Class

Define the OutputParser class to handle the parsing of the model's output

In [12]:
class OutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> AgentAction | AgentFinish:
            return AgentFinish({"output": text}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"

# initialize output parser for agent
parser = OutputParser()

## Initialize the Agent

Next, initialize the agent with the defined parser, memory buffer, tools, and the Konko model

In [13]:
# initialize agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    agent_kwargs={"output_parser": parser}
)

# Set Up the Conversation Flow

Konko manages special instruction tokens, simplifying the creation of LLM-compatible prompts and streamlining your chatbot's interactions, allowing you to focus on other important aspects of your chatbot design.

In [14]:
human_msg = "{input}"

agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

## Now we can begin asking questions...

In [15]:
output = agent("Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool.")



> Entering new AgentExecutor chain...
 Hi there! I'd be happy to help you learn how to use the crop tool in our photo editing app! 📸

To start, the crop tool is used to select a specific portion of an image and discard the rest. This can be useful when you want to focus on a particular subject or object within the image, or when you want to remove unwanted elements from the edges of the photo. 💡

To access the crop tool, you can find it in the top toolbar of the app. It looks like a small square with a diagonal line through it. When you click on the crop tool, you'll see a rectangular box appear around the image. 📐

To crop the image, simply drag the corners of the rectangular box to select the desired portion of the image. You can also use the handles on the sides of the box to adjust the size and position of the crop. 🔀

For example, if you want to crop a photo to focus on a specific subject, you can use the crop tool to select the portion of the image that contains the subject. Th

In [16]:
agent("I don't know how to adjust the size and position of the crop ")



> Entering new AgentExecutor chain...
 Sure thing! Adjusting the size and position of the crop is easy peasy. 🐰

To adjust the size of the crop, you can drag the handles on the sides of the rectangular box. The handles are the little lines that extend from the corners of the box. 🔀

For example, if you want to make the crop box smaller, you can drag the handles inward towards the center of the image. Conversely, if you want to make the crop box larger, you can drag the handles outward away from the center of the image. 📈

To adjust the position of the crop, you can drag the entire rectangular box to a different location within the image. For instance, if you want to crop a specific object within the image, you can drag the crop box to position it over the object. 📍

Remember, as you adjust the size and position of the crop, the preview of the cropped image will update in real-time, so you can see exactly what the final result will look like. 🔍

I hope that helps! Do you have any othe

{'input': "I don't know how to adjust the size and position of the crop ",
 'chat_history': [HumanMessage(content="Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool.", additional_kwargs={}, example=False),
  AIMessage(content=' Hi there! I\'d be happy to help you learn how to use the crop tool in our photo editing app! 📸\n\nTo start, the crop tool is used to select a specific portion of an image and discard the rest. This can be useful when you want to focus on a particular subject or object within the image, or when you want to remove unwanted elements from the edges of the photo. 💡\n\nTo access the crop tool, you can find it in the top toolbar of the app. It looks like a small square with a diagonal line through it. When you click on the crop tool, you\'ll see a rectangular box appear around the image. 📐\n\nTo crop the image, simply drag the corners of the rectangular box to select the desired portion of the image. You can als

In [17]:
agent("Thanks that worked! Could you also guide me on how to adjust the brightness and contrast of an image obtained above?")



> Entering new AgentExecutor chain...
 Of course! Adjusting the brightness and contrast of an image is a great way to enhance its overall look and feel. 💡

To adjust the brightness and contrast of an image, you can use the "Adjust" tool, which is located in the top toolbar of the app. The Adjust tool looks like a small gear icon. 🔧

When you click on the Adjust tool, a menu will appear with a variety of options for adjusting the brightness and contrast of the image. Here are the basic steps:

1. Click on the Adjust tool in the top toolbar.
2. Select the "Brightness" option from the menu.
3. Use the slider to adjust the brightness of the image. Moving the slider to the right will make the image brighter, while moving it to the left will make the image darker. 🌟
4. Select the "Contrast" option from the menu.
5. Use the slider to adjust the contrast of the image. Moving the slider to the right will increase the contrast, while moving it to the left will decrease the contrast. 🔩

As you 

{'input': 'Thanks that worked! Could you also guide me on how to adjust the brightness and contrast of an image obtained above?',
 'chat_history': [HumanMessage(content="Hi, I just downloaded your photo editing app, but I'm having trouble understanding how to use the crop tool.", additional_kwargs={}, example=False),
  AIMessage(content=' Hi there! I\'d be happy to help you learn how to use the crop tool in our photo editing app! 📸\n\nTo start, the crop tool is used to select a specific portion of an image and discard the rest. This can be useful when you want to focus on a particular subject or object within the image, or when you want to remove unwanted elements from the edges of the photo. 💡\n\nTo access the crop tool, you can find it in the top toolbar of the app. It looks like a small square with a diagonal line through it. When you click on the crop tool, you\'ll see a rectangular box appear around the image. 📐\n\nTo crop the image, simply drag the corners of the rectangular box 